# Sistema de recomendación de anime

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd 
from sklearn.neighbors import NearestNeighbors

El objetivo es aplicar todo lo que hemos aprendido hasta el momento, especialmente sobre gradiente descendiente. Para esto desarrollaremos un sistema de recomendación usando factorización de matrices.

En esta entrega se darán todos elementos necesarios para desarrollar un sistema de recomendación de amines, el objetivo es que usted entienda como funciona y desarrolle una solución.

Si desea conocer mas sobre el dataset que usaremos puede revisar acá: https://www.kaggle.com/CooperUnion/anime-recommendations-database

1- Cargue las bases de datos de calificaciones, usuarios y animes. Puede usar para este objetivo la biblioteca pandas. Genere el conjunto de entrenamiento usando los usuarios cómo filas y los animes cómo columnas. Tenga en cuenta que para el algoritmo es necesario que los -1 sean cero, sin embargo, para la recomendación si debe tener en cuenta dichos valores.

In [2]:
ratings_df = pd.read_csv('rating.csv', nrows=73516)
#anime_id = ratings_df['anime_id']
#user_id = ratings_df['user_id']
#rating  = ratings_df['rating']
#anime_df = pd.read_csv('anime.csv'

'''
i_limit, j_limit = training.shape
for i in range(0, i_limit):
    for j in range(0, j_limit):
        if rating[j] != -1:
           < training[i,j] = rating[j]
matrix_df = pd.pivot_table(ratings_df, values='rating', index='user_id', columns='anime_id')

matrix_df.head()
'''
matrix = np.array(pd.pivot_table(ratings_df, values='rating', index='user_id', columns='anime_id').
                  fillna(0).replace(-1, 0))
matrix_df = pd.pivot_table(ratings_df, values='rating', index='user_id', columns='anime_id').fillna(0).replace({-1:0})

print(matrix_df.head())

print(ratings_df.head())



anime_id  1      5      6      7      8      15     16     17     18     \
user_id                                                                   
1           0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2           0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
3           0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
4           0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
5           0.0    0.0    8.0    0.0    0.0    6.0    0.0    6.0    6.0   

anime_id  19     ...  33522  33524  33558  33569  33618  33659  33934  33964  \
user_id          ...                                                           
1           0.0  ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2           0.0  ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
3           0.0  ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
4           0.0  ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0

2- Agregue la función donde se ejecuta el algoritmo de factorización de matrices. Puede basarse en el algoritmo compartido en las dispositivas. No debe de gastar todas las interaciones. 

In [3]:
def do_matrix_factorization(R, P, Q, K, steps=5000, alpha=0.0002, beta=0.02):
    Q = Q.T
    for step in range(steps):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i,j] > 0:
                    eij = R[i,j] - P[i,:].dot(Q[:,j])
                    for k in range(K):
                        P[i,k] = P[i,k] + alpha * (2 * eij * Q[k,j] - beta * P[i,k])
                        Q[k,j] = Q[k,j] + alpha * (2 * eij * P[i,k] - beta * Q[k,j])
        eR = P.dot(Q)
        e = 0
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i,j] > 0:
                    e = e + pow(R[i,j] - P[i,:].dot(Q[:,j]), 2)
                    for k in range(K):
                        e = e + (beta/2) * (pow(P[i,k],2) + pow(Q[k,j],2))
        if e < 0.001:
            break
    return P, Q.T



3- Programe una función para generar $\hat{R}$. Recuerde que para generar $\hat{R}$ debe realizar los siguiente pasos:

    - Generar R usando el algoritmo de vecinos más cercanos.
    - Defina K.
    - Defina P y Q.
    - Generar R prima.


4- Realice las siguiente recomendaciones (animes que no ha visto con calificación 10):

    1- Usuario de id 51, películas.
    2- Usuario de id 510, animes con menos de 31 episodios.
    3- Usuario de id 17000, animes con rankins mayores o iguales a 9.
    4- Usuario de id 27005, animes de comedia y aventura.
    5- Usuario de id 37502, TV.
    6- Usuario de id 57502, películas y OVAS.
    7- Usuario de id 67501.

Cada recomendación debe tener como respuesta, número de iteraciones, número de recomendaciones y nombre de los animes,